In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging
import joblib

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
def create_train_test(X, order_by, training_size, testing_size, sample_size=10000):

    X = X.sort_values(by=order_by)
    sample = X.head(sample_size)
    
    X_train = sample.head(training_size)
    X_test = sample.tail(testing_size)
    
    return X_train, X_test

In [6]:
reviews_train, reviews_sub = create_train_test(reviews, 'date', 8000, 2000)

In [7]:
reviews_sub = reviews_train[["user_id", "movie_id", "rating"]]
print(reviews_sub.shape)
reviews_sub.head()

(8000, 3)


,user_id,movie_id,rating
498923,37287,2171847,6
442554,33140,444778,8
81920,6338,1411238,6
584570,43691,1496422,7
450669,33799,118799,5


In [8]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dec = FunkSVD(iters=20, learning_rate=0.005)
dec.fit(movie_reviews)

DEBUG:root:Optimization Statistics
DEBUG:root:Iterations | Mean Squared Error 
DEBUG:root:1 		 32.65673044368389
DEBUG:root:2 		 21.351377069829642
DEBUG:root:3 		 15.911932020763897
DEBUG:root:4 		 12.44794402115401
DEBUG:root:5 		 10.005153974948348
DEBUG:root:6 		 8.19842465767776
DEBUG:root:7 		 6.812434159541881
DEBUG:root:8 		 5.717378381578071
DEBUG:root:9 		 4.832190761936454
DEBUG:root:10 		 4.1047764491481225
DEBUG:root:11 		 3.500544943427475
DEBUG:root:12 		 2.99543051586739
DEBUG:root:13 		 2.5717550774444145
DEBUG:root:14 		 2.2158660298152455
DEBUG:root:15 		 1.9168033580943824
DEBUG:root:16 		 1.6655286077262021
DEBUG:root:17 		 1.4544536520881393
DEBUG:root:18 		 1.2771360991922365
DEBUG:root:19 		 1.1280761238842425
DEBUG:root:20 		 1.0025783657792964


FunkSVD(iters=20, learning_rate=0.005)

In [10]:
dec.X_transformed.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,4.995352,6.453561,4.715510,5.271581,8.246069,6.713653,6.329919,7.434965,6.322322,5.478906,...,7.320326,7.999544,6.899924,7.030752,2.364766,7.593153,8.693357,8.288268,9.077183,6.329699
46,4.716073,6.139984,3.824087,5.432235,7.525485,6.955170,6.658338,7.025797,6.275725,5.732467,...,6.287488,6.589302,6.648639,6.190661,2.375198,6.449824,9.433789,7.555151,7.451368,5.877744
48,4.027748,5.814584,3.480185,5.227274,6.304622,5.550424,5.248941,6.161061,4.753893,4.503053,...,5.406941,6.105846,6.616863,5.617021,1.672777,6.018252,5.886418,5.979838,7.094849,5.468404
51,3.968966,5.218073,3.853912,4.429536,6.607725,5.424027,4.640975,5.821117,4.954659,4.230619,...,6.338786,6.638174,6.072319,6.041385,1.748626,6.904062,6.310390,6.683005,7.523555,5.338106
66,4.157880,5.800857,2.553528,6.000931,6.229114,7.074606,6.394292,6.174211,5.726271,5.678796,...,5.104713,4.735809,7.057719,5.343449,2.051842,5.641657,8.843480,6.150761,5.371715,5.520331


In [11]:
dec.X_transformed.loc[8].sort_values(ascending=False).head(5).index.tolist()

[1255953, 1623205, 54215, 1675434, 1707386]

In [12]:
user_ids = [8, 46, 48, 51, 66]
dec.recommend(user_ids)

[[8, [1255953, 1623205, 54215, 1675434, 1707386]],
 [46, [1255953, 1045658, 1853728, 1675434, 2388725]],
 [48, [1255953, 54215, 108052, 1853728, 421715]],
 [51, [1623205, 1255953, 54215, 1707386, 1675434]],
 [66, [1255953, 1024648, 1045658, 68646, 454876]]]

In [14]:
joblib.dump(dec, "models/funk_svd.pkl")

['models/funk_svd.pkl']